In [5]:
#!pip install --upgrade pika

In [4]:
import pika
pika.__version__

'1.0.0'

In [6]:
#!sudo rabbitmqctl add_user myguest myguestpwd
#!sudo rabbitmqctl set_permissions -p / myguest "." "." ".*"

Creating user "myguest" ...
Setting permissions for user "myguest" in vhost "/" ...


In [7]:
!sudo rabbitmqctl list_users

Listing users ...
guest	[administrator]
myguest	[]


In [8]:
#connection = pika.BlockingConnection(
#    pika.ConnectionParameters(host='localhost'))
#channel = connection.channel()

In [9]:
credentials = pika.PlainCredentials('myguest', 'myguestpwd')
#URL='localhost'
URL='PROD-JOB-844fd7d2202ac4da.elb.us-east-2.amazonaws.com'
parameters =  pika.ConnectionParameters(URL, port=5672, credentials=credentials)
connection = pika.BlockingConnection(parameters)
channel = connection.channel()

In [10]:
def fib(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n - 1) + fib(n - 2)

In [11]:
def on_request(ch, method, props, body):
    n = int(body)

    print(" [.] fib(%s)" % n)
    response = fib(n)

    ch.basic_publish(exchange='',
                     routing_key=props.reply_to,
                     properties=pika.BasicProperties(correlation_id = \
                                                         props.correlation_id),
                     body=str(response))
    ch.basic_ack(delivery_tag=method.delivery_tag)


In [12]:
channel.queue_declare(queue='rpc_queue')
channel.basic_qos(prefetch_count=1)
channel.basic_consume(queue='rpc_queue', on_message_callback=on_request)

'ctag1.ab582b3ca0304a358a0752c41b1bd673'

In [ ]:
print(" [x] Awaiting RPC requests")
channel.start_consuming()

 [x] Awaiting RPC requests
 [.] fib(30)
 [.] fib(20)
 [.] fib(32)
 [.] fib(14)
